In [1]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable, Optional
from datetime import datetime, timedelta
from IPython.display import display

import numpy as np
import pandas as pd

from sesto.constants import CURRENCIES, MT5Timeframe
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.backtester import Backtester, Trade

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 8
Number of pairs: 56


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAMES = [MT5Timeframe.M5, MT5Timeframe.M15, MT5Timeframe.H1, MT5Timeframe.H4]
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 19)

MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

In [5]:
BACKTEST_FROM_DATE = datetime(2024, 1, 1)
BACKTEST_TO_DATE = datetime(2024, 9, 20)
MAIN_TIMEFRAME = MT5Timeframe.H1

INITIAL_CAPITAL = 10000

CSM_TREND_LONG_THRESHOLD = 65
CSM_TREND_SHORT_THRESHOLD = 35
CSM_REVERSAL_LONG_THRESHOLD = 85
CSM_REVERSAL_SHORT_THRESHOLD = 15
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
ATR_VOLATILITY_THRESHOLD = 0.02

CAPITAL_PER_TRADE = 200  # $200 per trade

RR_MULTIPLIER = 2
TRADE_SL_MULTIPLIER = 0.0025

TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001

LEVERAGE = 500

In [6]:
mtd.fetch_data_all_timeframes(PAIRS, TIMEFRAMES, DATA_FROM_DATE, DATA_TO_DATE)

Failed to fetch data for symbol: USDEUR on timeframe: MT5Timeframe.M5
No data fetched for pair: USDEUR on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDGBP on timeframe: MT5Timeframe.M5
No data fetched for pair: USDGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: USDJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCAD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDAUD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDAUD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDNZD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDNZD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURUSD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: EURJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCAD on timeframe: MT5T

In [7]:
# Remove symbols with no data for any timeframe
for timeframe in TIMEFRAMES:
    empty_symbols = [symbol for symbol, df in mtd.data[timeframe].items() if df is None or df.empty]
    for symbol in empty_symbols:
        del mtd.data[timeframe][symbol]
        if symbol in PAIRS:
            PAIRS.remove(symbol)

print(f"Remaining pairs after removing empty data: {len(PAIRS)}")

Remaining pairs after removing empty data: 28


In [8]:
import plotly.graph_objects as go

def plot_data_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                actual_rows = len(mtd.data[timeframe][symbol])
                normalized_value = actual_rows / expected_rows[timeframe]
                heatmap_data.append({
                    'Timeframe': timeframe.name,
                    'Symbol': symbol,
                    'Normalized Row Count': normalized_value
                })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Row Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Row Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Row Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Row Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_data_availability()

In [9]:
from sesto.indicators import SMA, EMA, RSI, ROC, MACD, BB, ATR

for timeframe, pairs_data in mtd.data.items():
    for pair, df in pairs_data.items():
        if df is not None and not df.empty:
            for period in MA_PERIODS:
                SMA(df, period)
                EMA(df, period)
                ROC(df, period)
                RSI(df, period)
                BB(df, period, 2)
                ATR(df, period)
    
            df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

display(mtd.data[MT5Timeframe.H1]['EURUSD'].head())



,time,open,high,low,close,tick_volume,spread,real_volume,sma-7,ema-7,...,bbu-14-2,bbl-14-2,sma-21,ema-21,roc-21,rsi-21,bbm-21-2,bbu-21-2,bbl-21-2,psm
0,2024-04-04 21:00:00,1.08600,1.08600,1.08466,1.08500,2883,0,0,NaN,1.085000,...,NaN,NaN,NaN,1.085000,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-04 22:00:00,1.08499,1.08507,1.08333,1.08336,3453,0,0,NaN,1.084590,...,NaN,NaN,NaN,1.084851,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-04 23:00:00,1.08336,1.08381,1.08315,1.08362,1028,0,0,NaN,1.084348,...,NaN,NaN,NaN,1.084739,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-05 00:00:00,1.08361,1.08397,1.08229,1.08373,2068,15,0,NaN,1.084193,...,NaN,NaN,NaN,1.084647,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-05 01:00:00,1.08367,1.08396,1.08352,1.08360,659,7,0,NaN,1.084045,...,NaN,NaN,NaN,1.084552,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import plotly.graph_objects as go
import pandas as pd

def plot_indicator_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                df = mtd.data[timeframe][symbol]
                if df is not None and not df.empty:
                    # Check for the presence of indicators
                    indicator_columns = [col for col in df.columns if col.startswith(('sma-', 'ema-', 'rsi-', 'roc-', 'macd', 'bb'))]
                    if indicator_columns:
                        actual_rows = df[indicator_columns].dropna().shape[0]
                        normalized_value = actual_rows / expected_rows[timeframe]
                        heatmap_data.append({
                            'Timeframe': timeframe.name,
                            'Symbol': symbol,
                            'Normalized Indicator Count': normalized_value
                        })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Indicator Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Indicator Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Indicator Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Indicator Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_indicator_availability()

In [11]:
import pandas_ta as ta

for timeframe in TIMEFRAMES:
    for currency in CURRENCIES:
        # Initialize DataFrames to hold PSM values for the current currency as base and quote
        base_psm_df = pd.DataFrame()
        quote_psm_df = pd.DataFrame()
        
        # Iterate over all pair data for the current timeframe
        for symbol, df in mtd.data[timeframe].items():
            if df is None or df.empty:
                continue
            
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency:
                # Use PSM as is for base currency
                base_psm_df[symbol] = df['psm']
                base_psm_df['time'] = df['time']
            elif currency == quote_currency:
                # Invert PSM for quote currency
                quote_psm_df[symbol] = -df['psm']
                quote_psm_df['time'] = df['time']
        
        # Calculate base CSM
        if not base_psm_df.empty:
            base_csm_df = base_psm_df.groupby('time').mean()
            base_csm_df['csm'] = base_csm_df.mean(axis=1)
            base_csm_df['csm'] = base_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = base_csm_df['csm'].min()
            max_csm = base_csm_df['csm'].max()
            base_csm_df['csm'] = 100 * (base_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for base CSM
            MACD(base_csm_df, 'csm', 12, 26, 9)
        
        # Calculate quote CSM
        if not quote_psm_df.empty:
            quote_csm_df = quote_psm_df.groupby('time').mean()
            quote_csm_df['csm'] = quote_csm_df.mean(axis=1)
            quote_csm_df['csm'] = quote_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = quote_csm_df['csm'].min()
            max_csm = quote_csm_df['csm'].max()
            quote_csm_df['csm'] = 100 * (quote_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for quote CSM
            MACD(quote_csm_df, 'csm', 12, 26, 9)
        
        # Add CSM and MACD values to all relevant symbols in mtd.data
        for symbol in mtd.data[timeframe]:
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency and not base_csm_df.empty:
                mtd.data[timeframe][symbol]['base_csm'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['csm'])
                mtd.data[timeframe][symbol]['base_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd'])
                mtd.data[timeframe][symbol]['base_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['base_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['base_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['base_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)

            elif currency == quote_currency and not quote_csm_df.empty:

                mtd.data[timeframe][symbol]['quote_csm'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['csm'])
                mtd.data[timeframe][symbol]['quote_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd'])
                mtd.data[timeframe][symbol]['quote_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['quote_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['quote_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['quote_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)



In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

def plot_random_data():
    # Choose a random timeframe and symbol
    random_timeframe = random.choice(TIMEFRAMES)
    random_symbol = random.choice(PAIRS)

    # Get the data for the chosen timeframe and symbol
    df = mtd.data[random_timeframe][random_symbol]

    # Create subplots
    fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=('Close Price + SMA', 'CSM Values', 'Base CSM MACD', 'Quote CSM MACD', 'RSI', 'ROC'))

    # 1. Close Price + SMA
    fig.add_trace(go.Scatter(x=df['time'], y=df['close'], name='Close Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['sma-14'], name='SMA-14'), row=1, col=1)

    # 2. CSM Values
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm'], name='Base CSM'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm'], name='Quote CSM'), row=2, col=1)

    # 3. Base CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd'], name='Base MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd_signal'], name='Base Signal'), row=3, col=1)
    fig.add_bar(x=df['time'], y=df['base_csm_macd_histogram'], name='Base Histogram', row=3, col=1)

    # 4. Quote CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd'], name='Quote MACD'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd_signal'], name='Quote Signal'), row=4, col=1)
    fig.add_bar(x=df['time'], y=df['quote_csm_macd_histogram'], name='Quote Histogram', row=4, col=1)

    # 5. RSI
    fig.add_trace(go.Scatter(x=df['time'], y=df['rsi-14'], name='RSI-14'), row=5, col=1)

    # 6. ROC
    fig.add_trace(go.Scatter(x=df['time'], y=df['roc-14'], name='ROC-14'), row=6, col=1)

    # Update layout
    fig.update_layout(
        height=1500, 
        width=1600, 
        title_text=f"Chart for {random_symbol} ({random_timeframe.name})", 
        template='plotly_dark'
    )

    # Update x-axis to include range slider
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        row=6, col=1  # Apply to the bottom subplot
    )

    # Hide rangeslider for all subplots except the bottom one
    for i in range(1, 6):
        fig.update_xaxes(rangeslider_visible=False, row=i, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="CSM", row=2, col=1)
    fig.update_yaxes(title_text="Base MACD", row=3, col=1)
    fig.update_yaxes(title_text="Quote MACD", row=4, col=1)
    fig.update_yaxes(title_text="RSI", row=5, col=1)
    fig.update_yaxes(title_text="ROC", row=6, col=1)

    # Show the plot
    fig.show()

# plot_random_data()

In [13]:
class MyStrategy(Backtester):    
    def entry_condition(
        self, 
        symbol: str, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe,
    ) -> Optional[Dict]:
        # Check if we already have an open trade for this symbol
        if any(trade.symbol == symbol for trade in open_trades):
            return None

        base_csm = row.get('base_csm')
        quote_csm = row.get('quote_csm')
    
        trend_up = (
            base_csm > CSM_TREND_LONG_THRESHOLD and 
            quote_csm < CSM_TREND_SHORT_THRESHOLD
        )

        trend_down = (
            base_csm < CSM_TREND_SHORT_THRESHOLD and 
            quote_csm > CSM_TREND_LONG_THRESHOLD
        )

        entry = None

        if trend_up:
            entry_price = row['close'] * ( 1 + SLIPPAGE)

            entry = {
                'entry_price': entry_price,
                'position_type': 'long',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp': entry_price * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl': entry_price * (1 - TRADE_SL_MULTIPLIER)
            }
        elif trend_down:
            entry_price = row['close'] * ( 1 - SLIPPAGE)

            entry = {
                'entry_price': entry_price,
                'position_type': 'short',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp': entry_price * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl': entry_price * (1 + TRADE_SL_MULTIPLIER)
            }

        # Ensure no open trades for the symbol
        if entry and not any(trade.symbol == symbol for trade in open_trades):
            return entry

        return None

    def exit_condition(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ) -> bool:
        return False

    def trailing_stop(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ):
        pass

In [14]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    main_timeframe=MAIN_TIMEFRAME,
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=LEVERAGE
)

backtest.run()

Opened long trade for USDJPY at 155.6785663. Available capital: 9800.00
Closed long trade for USDJPY at 156.45695913149999 with PNL 469.95. Available capital: 10469.95
Opened short trade for USDJPY at 155.1274857. Available capital: 10269.95
Closed short trade for USDJPY at 154.3518482715 with PNL 470.05. Available capital: 10940.00
Opened short trade for USDJPY at 154.0505934. Available capital: 10740.00
Closed short trade for USDJPY at 153.280340433 with PNL 470.05. Available capital: 11410.05
Opened short trade for USDJPY at 153.0386946. Available capital: 11210.05
Closed short trade for USDJPY at 153.42129133650002 with PNL -280.02. Available capital: 11130.02
Opened short trade for USDJPY at 153.6816303. Available capital: 10930.02
Closed short trade for USDJPY at 152.91322214849998 with PNL 470.05. Available capital: 11600.08
Opened short trade for USDJPY at 152.8237161. Available capital: 11400.08
Closed short trade for USDJPY at 153.20577539025 with PNL -280.03. Available capit

In [15]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log

,symbol,entry_time,entry_price,position_type,position_size_usd,tp,sl,used_capital,potential_profit_usd,potential_profit_percent,...,potential_loss_percent,tp_price_diff_percent,sl_price_diff_percent,close_time,close_price,pnl,max_drawdown,max_profit,closing_reason,unrealized_pnl
0,USDJPY,2024-04-25 09:00:00,155.678566,long,100000,156.456959,155.289370,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-04-26 10:00:00,156.441313,469.950000,0,0,tp,0
1,USDJPY,2024-05-02 11:00:00,155.127486,short,100000,154.351848,155.515304,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-05-02 17:00:00,154.367283,470.050000,0,0,tp,0
2,USDJPY,2024-05-02 17:00:00,154.050593,short,100000,153.280340,154.435720,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-05-02 21:00:00,153.295668,470.050000,0,0,tp,0
3,USDJPY,2024-05-02 21:00:00,153.038695,short,100000,152.273501,153.421291,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-05-02 23:00:00,153.436633,-280.025000,0,0,sl,0
4,USDJPY,2024-05-02 23:00:00,153.681630,short,100000,152.913222,154.065834,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-05-03 04:00:00,152.928513,470.050000,0,0,tp,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,AUDJPY,2024-09-18 18:00:00,95.919591,long,100000,96.399189,95.679792,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-09-18 20:00:00,96.016397,80.924534,0,0,end_of_backtest,0
1356,AUDCHF,2024-09-18 07:00:00,0.570567,long,100000,0.573420,0.569141,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-09-18 20:00:00,0.571333,114.219107,0,0,end_of_backtest,0
1357,NZDJPY,2024-09-18 10:00:00,88.171816,long,100000,88.612675,87.951387,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-09-18 20:00:00,88.155184,-38.863964,0,0,end_of_backtest,0
1358,NZDCHF,2024-09-18 05:00:00,0.524712,long,100000,0.527336,0.523401,200.0,500.0,250.0,...,125.0,0.5,0.25,2024-09-18 20:00:00,0.524648,-32.375540,0,0,end_of_backtest,0


In [16]:
performance_report = backtest.generate_report()
performance_report

,Metric,Value
0,Initial Capital,$10000.00
1,Final Capital,$38181.26
2,Total Profit,$28181.26
3,Return (%),281.81%
4,Annualized Return (%),1908.70%
5,Volatility (Ann.),372.55
6,Sharpe Ratio,5.12
7,Sortino Ratio,10.10
8,Calmar Ratio,25.05
9,Max. Drawdown ($),$-7620.99
